In [1]:
import pandas as pd
from sklearn.decomposition import PCA
import sys

sys.path.append("../")
from adelle.utils import (
    model_generator,
    evaluate,
    get_Xy,
    preprocess,
    create_pca_analysis_pipeline,
    analyze_pca_feature_importance,
    preprocess_and_convert_to_df,
    print_linear_regression_pvalues,
    print_logistic_regression_pvalues,
)

In [2]:
df = pd.read_csv("../data/predictive_maintenance.csv")

In [3]:
df

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure
...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,No Failure
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,No Failure
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,No Failure
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,No Failure


In [4]:
X_train, X_test, y_train, y_test = get_Xy(
    df, "Target", drop_cols=["UDI", "Product ID", "Failure Type"]
)

In [5]:
print(X_train)

     Type  Air temperature [K]  Process temperature [K]  \
8045    M                300.7                    311.9   
4965    M                303.8                    312.7   
9912    L                298.6                    308.9   
8247    L                299.0                    310.4   
4938    M                303.5                    312.4   
...   ...                  ...                      ...   
2414    L                299.1                    308.5   
1098    M                296.9                    307.4   
6836    H                301.1                    311.0   
6819    L                300.9                    310.7   
4673    L                303.4                    311.5   

      Rotational speed [rpm]  Torque [Nm]  Tool wear [min]  
8045                    1366         55.1               36  
4965                    1364         49.3              167  
9912                    1327         58.8                9  
8247                    1713         28.0      

In [6]:
preprocessor = preprocess(X_train, y_train, label_cols=["Target"])

In [7]:
pca_pipeline = create_pca_analysis_pipeline(
    X_train, y_train, label_cols=None, n_components=5
)
analyze_pca_feature_importance(X_train, pca_pipeline)

Top contributing features by component:
Component 1:
  ss_encode_X__Torque [Nm]: -0.996
  minmax_encode_X__Rotational speed [rpm]: 0.092
  ohe__if-Type-c-M: -0.003
  ohe__if-Type-c-L: 0.003
  minmax_encode_X__Process temperature [K]: 0.002

Component 2:
  ohe__if-Type-c-L: -0.742
  ohe__if-Type-c-M: 0.666
  ohe__if-Type-c-H: 0.076
  ss_encode_X__Torque [Nm]: -0.004
  minmax_encode_X__Tool wear [min]: -0.003

Component 3:
  ohe__if-Type-c-H: 0.812
  ohe__if-Type-c-M: -0.472
  ohe__if-Type-c-L: -0.340
  minmax_encode_X__Air temperature [K]: -0.034
  minmax_encode_X__Process temperature [K]: -0.026

Component 4:
  minmax_encode_X__Air temperature [K]: -0.773
  minmax_encode_X__Process temperature [K]: -0.627
  minmax_encode_X__Tool wear [min]: -0.088
  ohe__if-Type-c-H: -0.034
  ohe__if-Type-c-M: 0.020

Component 5:
  minmax_encode_X__Tool wear [min]: 0.996
  minmax_encode_X__Air temperature [K]: -0.070
  minmax_encode_X__Process temperature [K]: -0.054
  ohe__if-Type-c-H: -0.007
  ohe__i

In [8]:
X_train_preprocessed_df, y_train_preprocessed = preprocess_and_convert_to_df(
    X_train, y_train, preprocessor
)

ValueError: No valid specification of the columns. Only a scalar, list or slice of all integers or all strings, or boolean mask is allowed

In [ ]:
print_linear_regression_pvalues(X_train_preprocessed_df, y_train_preprocessed)

ValueError: The indices for endog and exog are not aligned

In [ ]:
print_logistic_regression_pvalues(X_train_preprocessed_df, y_train_preprocessed)